In [1]:
import matplotlib.pyplot as plt
import numpy as np
import keras
import tensorflow as tf
import os
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adadelta
from keras.utils import np_utils
import numpy
import csv
import scipy.misc
import scipy
from scipy import ndimage
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

c:\users\kevin\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [3]:
loaded_model

In [4]:
import cv2
import sys

# Get user supplied values
# imagePath = sys.argv[1]
imagePath = 'images/kevin-sad-new.jpg'
cascPath = "haarcascade_frontalface_default.xml"

# Create the haar cascade
faceCascade = cv2.CascadeClassifier(cascPath)

# Read the image
image = cv2.imread(imagePath)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


# Detect faces in the image
faces = faceCascade.detectMultiScale(
    gray,
    scaleFactor=1.1,
    minNeighbors=5,
    minSize=(30, 30),
)

print("Found {0} faces!".format(len(faces)))

# Draw a rectangle around the faces
for (x, y, w, h) in faces:
    print ('x, y =', (x, y))
    print ('w, h =', (w, h))
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
    print ('x+w = ', x+w)
    print ('y+h = ', y+h)
#     cv2.imwrite('images/kevin-happy-found.jpg', img[x:x+w, y:y+h])

cv2.imshow("img", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Found 1 faces!
x, y = (508, 204)
w, h = (310, 310)
x+w =  818
y+h =  514


In [5]:
import cv2
img = cv2.imread('images/kevin-sad-new.jpg')
crop_img = img[y:y+h, x:x+w]
cv2.imwrite('images/kevin-sad-new-cropped.jpg', crop_img)
cv2.imshow("cropped", crop_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
import os
import cv2
import PIL

print (os.getcwd())
image = cv2.imread('images/kevin-sad-new-cropped.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray

E:\GT\DeepLearning_Hackathon


array([[220, 219, 216, ..., 127, 129, 129],
       [220, 219, 216, ..., 126, 129, 130],
       [221, 218, 215, ..., 127, 130, 132],
       ...,
       [ 65,  32,  51, ...,  53,  55,  56],
       [ 57,  31,  48, ...,  54,  56,  56],
       [ 55,  33,  48, ...,  55,  56,  57]], dtype=uint8)

In [7]:
cv2.imshow("img", gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
from PIL import Image

img = cv2.resize(gray, (48, 48), PIL.Image.ANTIALIAS)

In [9]:
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
preds = loaded_model.predict(img.reshape(1, 48, 48, 1))

In [11]:
preds

array([[0.08941597, 0.00169639, 0.03757307, 0.00144573, 0.09324053,
        0.00492798, 0.7717003 ]], dtype=float32)

In [12]:
mapping = {0: 'Angry',
 1: 'Disgust',
 2: 'Fear',
 3: 'Happy',
 4: 'Sad',
 5: 'Surprise',
 6: 'Neutral'}

In [13]:
np.argmax(preds)

6

In [14]:
correct_answer = 4

In [15]:
preds[0][correct_answer] * 10

0.9324052929878235

# Unit tests

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import keras
import tensorflow as tf
import os
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adadelta
from keras.utils import np_utils
import scipy.misc
import scipy
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json
import cv2
import sys
import PIL
from PIL import Image

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

def load(model_loc, weights_loc):
    json_file = open(model_loc, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(weights_loc)
    print(type(loaded_model))
    return loaded_model


def bounding_boxes(image_loc):
    imagePath = image_loc
    cascPath = "haarcascade_frontalface_default.xml"

    # Create the haar cascade
    faceCascade = cv2.CascadeClassifier(cascPath)

    # Read the image
    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


    # Detect faces in the image
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
    )

    print("Found {0} faces!".format(len(faces)))

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

    cv2.imshow("img", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return x, y, w, h

def crop(x, y, w, h):
    img = cv2.imread('images/kevin-sad-new.jpg')
    crop_img = img[y:y+h, x:x+w]
    cv2.imwrite('images/kevin-sad-new-cropped.jpg', crop_img)
    cv2.imshow("cropped", crop_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return crop_img

def gray(cropped_img):
    gray = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(gray, (48, 48), PIL.Image.ANTIALIAS)
    return img



loaded_model = load('model.json', 'model.h5')
x, y, w, h = bounding_boxes('images/kevin-sad-new.jpg')
cropped_img = crop(x, y, w, h)
img = gray(cropped_img)

preds = loaded_model.predict(img.reshape(1, 48, 48, 1))
mapping = {0: 'Angry',
 1: 'Disgust',
 2: 'Fear',
 3: 'Happy',
 4: 'Sad',
 5: 'Surprise',
 6: 'Neutral'}

## have the correct answer beforehand
correct_answer = 4

score = preds[0][correct_answer] * 10
print ('Score = ', score)

c:\users\kevin\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


<class 'keras.models.Sequential'>
Found 1 faces!
Score =  0.9187986701726913


In [28]:
type(loaded_model) == keras.models.Sequential

True

In [30]:
import unittest 

class MyTest(unittest.TestCase):
    def test(self):
        self.assertEqual(type(load('model.json', 'model.h5')), keras.models.Sequential)
tester = MyTest()

In [31]:
tester.test()

<class 'keras.models.Sequential'>
